In [17]:
# import requests
import requests
import json

In [18]:
# geo.admin lookup

def get_lat_lon_wgs84(address):
    url = f"https://api3.geo.admin.ch/rest/services/api/SearchServer?searchText={address}&type=locations&origins=address"
    response = requests.get(url)
    data = response.json()

    return data["results"][0]["attrs"]["lat"], data["results"][0]["attrs"]["lon"]

get_lat_lon_wgs84("St. Leonhard-Strasse 45, St. Gallen")

(47.42184829711914, 9.369429588317871)

In [24]:
def get_easting_northing_lv95(address):

    url = f"https://api3.geo.admin.ch//rest/services/api/SearchServer?lang=de&searchText={address}&type=locations&sr=2056"
    response = requests.get(url)
    data = response.json()

    return data["results"][0]["attrs"]["y"], data["results"][0]["attrs"]["x"]

get_easting_northing_lv95("St. Leonhard-Strasse 45, St. Gallen")

(2745681.0, 1254130.25)

In [25]:
# function to get heating info from long and lat

def get_heating_info(lat, lon, dist):
    url = f"https://daten.stadt.sg.ch/api/records/1.0/search/?dataset=warmeversorgung&q=&facet=waermevers&geofilter.distance={lat}%2C{lon}%2C{dist}"
    response = requests.get(url)
    data = response.json()
    return data

# function to get heating info from address

def get_heating_info_from_address(address, dist):
    lat, lon = get_lat_lon_wgs84(address)
    return get_heating_info(lat, lon, dist)

get_heating_info_from_address("St. Leonhard-Strasse 45, St. Gallen", 1)

{'nhits': 1,
 'parameters': {'dataset': 'warmeversorgung',
  'rows': 10,
  'start': 0,
  'facet': ['waermevers'],
  'format': 'json',
  'geofilter.distance': ['47.42184829711914,9.369429588317871,1'],
  'timezone': 'UTC'},
 'records': [{'datasetid': 'warmeversorgung',
   'recordid': '41a0060d5f24a87f0c0f390b4e363916bd084a21',
   'fields': {'geo_shape': {'coordinates': [[[9.369402092347727,
        47.42141803157732],
       [9.36880080458217, 47.421178234079115],
       [9.36845445807961, 47.42153135919113],
       [9.369074349371125, 47.421789138301264],
       [9.369639976357611, 47.42203562120369],
       [9.369902089204528, 47.42170847062191],
       [9.369460130185418, 47.421532842160765],
       [9.369402092347727, 47.42141803157732]]],
     'type': 'Polygon'},
    'waermevers': 'Fernwärme Ergänzungsgebiet',
    'geo_point_2d': [47.42160099351865, 9.369180665597083],
    'dist': '33.27289013497016'},
   'geometry': {'type': 'Point',
    'coordinates': [9.369180665597083, 47.42160

In [26]:
# function to get waermevers from address

def get_waermevers_from_address(address, dist):
    data = get_heating_info_from_address(address, dist)
    return data["records"][0]["fields"]["waermevers"]

get_waermevers_from_address("St. Leonhard-Strasse 45, St. Gallen", 1)

'Fernwärme Ergänzungsgebiet'

# data from geo.admin.ch

ch.bfe.solarenergie-eignung-daecher
ch.bfe.solarenergie-eignung-fassaden

# get data from geo.admin.ch

https://map.geo.admin.ch/?layers=ch.bfe.solarenergie-eignung-daecher&lang=en&topic=ech&bgLayer=ch.swisstopo.pixelkarte-farbe&E=2745681.00&N=1254117.67&zoom=11.371666666666666


In [47]:
# https://github.com/ThesisRocker/Data2DollarFS22/blob/0f7344650e6c6c43bc4057502493ea4afbce64f3/06_Abgabe%20Gruppenarbeit/Gruppe%201/sonnendach/script_solar_2.py

def get_geo_admin_data(address):
    addr_easting, addr_northing = get_easting_northing_lv95(address)

    # Build the request similar to script_solar only with new coordinates from previous request
    # Uses the swiss coordinate system LV95
    API_building_ID = ('https://api3.geo.admin.ch//rest/services/api/MapServer/identify'
                       '?geometryType=esriGeometryPoint'#&returnGeometry=true'
                       '&layers=all:ch.bfe.solarenergie-eignung-daecher'
                       '&geometry=' + str(addr_easting) + ',' + str(addr_northing) +
                       '&tolerance=0&order=distance&lang=de&sr=2056')


    url = f"https://api3.geo.admin.ch/rest/services/all/MapServer/identify?geometry={str(addr_easting)},{str(addr_northing)}&geometryFormat=geojson&geometryType=esriGeometryPoint&imageDisplay=1680,388,96&lang=en&layers=all:ch.bfe.solarenergie-eignung-daecher&limit=10&mapExtent=2745356.382965297,1254042.6972188372,2746005.609812927,1254192.6377050756&returnGeometry=true&sr=2056&tolerance=10"

    # Make the request to the API using requests library
    result_building = requests.get(url)

    # Transform result into json format
    result_building_json = result_building.json()

    print(result_building_json["results"][0])
    print(result_building_json["results"][1])

get_geo_admin_data("St. Leonhard-Strasse 45, St. Gallen")

{'type': 'Feature', 'featureId': 16745794, 'bbox': [2745660.3, 1254105.7, 2745705.5, 1254142.5], 'layerBodId': 'ch.bfe.solarenergie-eignung-daecher', 'layerName': 'Solar energy: suitability of roofs', 'id': 16745794, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[2745702.8, 1254126.4], [2745671.5, 1254107.1], [2745669.2, 1254105.7], [2745660.3, 1254120.1], [2745696.6, 1254142.5], [2745705.5, 1254128.1], [2745702.8, 1254126.4]], [[2745661.8, 1254119.4], [2745669.5, 1254106.9], [2745704.1, 1254128.3], [2745696.4, 1254140.8], [2745661.8, 1254119.4]]]]}, 'properties': {'monate': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 12, 11], 'building_id': 527069, 'a_param': [0.7768488835, 0.9652010208, 0.9679495756, 0.980280681, 0.9901538902, 0.9768872537, 0.9585564666, 0.9063184743, 0.7307145044, 0.3759406691, 0.3489994673, 0.5538456654], 'b_param': [0.8689646264, 0.9691674958, 1.0244670528, 0.9635810253, 0.9357137619, 0.9188822209, 0.9934626264, 0.9902676501, 0.9607837788, 0.8494011796, 0.8318482567,

In [49]:
def get_link_to_sonnendach(id):
    return f"https://www.uvek-gis.admin.ch/BFE/sonnendach/?featureId={id}&lang=de"

get_link_to_sonnendach(16745794)

'https://www.uvek-gis.admin.ch/BFE/sonnendach/?featureId=16745794&lang=de'